# Imports:

In [1]:
import pyarrow as pa 
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
from datetime import datetime

# Get Data:

In [2]:
dataset = pq.ParquetDataset('s3://ursa-labs-taxi-data/2009/', partitioning=["month"])

In [3]:
data_table = dataset.fragments[0].to_table()

# Questions:

### Can you get the average transaction between 2:00-2:59 PM

First we filter so that only transactions that started and ended at 2pm are available

In [4]:
pickup_hour = pc.hour(data_table['pickup_at'])
dropoff_hour = pc.hour(data_table['dropoff_at'])

pickup_time_filter = pc.equal(pickup_hour, 14)
dropoff_time_filter = pc.equal(dropoff_hour, 14)
combined_filter = pc.and_(pickup_time_filter, dropoff_time_filter)

filtered_pc = pc.filter(data_table, combined_filter)

Next we sum the transactions, then calculate the mean

In [5]:
added_transactions = pc.add(filtered_pc['fare_amount'],filtered_pc['tip_amount'])
pc.mean(added_transactions)

<pyarrow.DoubleScalar: 8.723216966053515>

This gives us a average transaction cost of 8.72

### Which day, on average has the highest tip? 

First we convert date times to days of the week

In [6]:
DOW_values = pc.day_of_week(data_table['pickup_at'])
DOW_values

[
  [
    6,
    6,
    5,
    3,
    5,
    ...
    1,
    1,
    4,
    5,
    4
  ]
]

* Then, we compare the values of each mean to find the highest one

In [7]:
highest_mean = 0
highest_day = 0
for i in range(7):
    day_filter = pc.equal(DOW_values, i)
    filtered_by_day = pc.filter(data_table,day_filter)
    current_mean = pc.mean(filtered_by_day['fare_amount'])
    if (pc.greater(current_mean,highest_mean)).as_py():
        highest_mean = current_mean
        highest_day = i
highest_day

6

In [8]:
highest_mean

<pyarrow.DoubleScalar: 9.792338970530873>

Therefore, we know that the day with the highest average cost is sunday (day 6) with an average cost of 9.79

### Which time (hour) of the day has the highest tip?

First we filter the data by the hour it was started in

In [9]:
by_hour = pc.hour(data_table['pickup_at'])

Then we find the hour with the highest average transaction cost

In [10]:
highest_mean = 0
highest_hour = 0
for i in range(24):
    hour_filter = pc.equal(by_hour, i)
    filtered_by_hour = pc.filter(data_table,hour_filter)
    current_mean = pc.mean(filtered_by_hour['tip_amount'])
    if (pc.greater(current_mean,highest_mean)).as_py():
        highest_mean = current_mean
        highest_hour = i
highest_hour

5

In [11]:
highest_mean

<pyarrow.DoubleScalar: 0.625723054615612>

Here we see that the hour with the highest average tip is 4 am (hour 5) with a tip value of 0.63